# Replication of VIX

This notebook shows how to reproduce the VIX given the data in CBOE White Paper (http://www.cboe.com/micro/vix/vixwhite.pdf). The code works for any option data set, not only one day as in the White Paper. The option data for this example is exactly the same as in the Appendix 1 of the White Paper.

Given are the prices $C_{i}$, $i\in\left\{ 0,\ldots,n\right\}$, of a series of European call options on the index with fixed maturity date $T$ and exercise prices $K_{i}$, $i\in\left\{ 0,\ldots,n\right\}$, as well as the prices $P_{i}$, $i\in\left\{ 0,\ldots,n\right\}$, of a series of European put options on the index with the same maturity date $T$ and exercise prices $K_{i}$. Let further hold $K_{i}<K_{i+1}$ for all $i\in\left\{ 0,\ldots,n-1\right\}$.

The VIX itself is
$$VIX=100\cdot\sqrt{V^{2}},$$
where $V$ is explained below.

Since there are days when there no options with precisely 30 days to expiration, we have to interpolate between near-term index and next-term index:
$$V^{2}=\left[T_{1}\sigma_{1}^{2}\left(\frac{N_{T_{2}}-N_{30}}{N_{T_{2}}-N_{T_{1}}}\right)+T_{2}\sigma_{2}^{2}\left(\frac{N_{30}-N_{T_{1}}}{N_{T_{2}}-N_{T_{1}}}\right)\right]\frac{365}{30}$$
with each $\sigma_{i}^{2}$ computed according to
$$\sigma^{2}=\frac{2}{T}\sum_{i=0}^{n}\frac{\Delta K_{i}}{K_{i}^{2}}e^{rT}M_{i}-\frac{1}{T}\left(\frac{F}{K_{*}}-1\right)^{2},$$
where the distance between strikes is
$$\Delta K_{i}	=	\begin{cases}
K_{1}-K_{0}, & i=0\\
\frac{1}{2}\left(K_{i+1}-K_{i-1}\right), & i=1,\ldots,n-1\\
K_{n}-K_{n-1}, & i=n
\end{cases}$$
the out-of-the-money option premium is
$$M_{i}	=	\begin{cases}
P_{i}, & K_{i}<K_{*}\\
\frac{1}{2}\left(P_{i}+C_{i}\right), & K_{i}=K_{*}\\
C_{i}, & K_{i}>K_{*}
\end{cases}$$
at-the-money strike price is
$$K_{*}	=	\max\left\{ K_{i}<F\right\},$$
forward price extracted from put-call parity:
$$F	=	K_{j}+e^{rT}\left|C_{j}-P_{j}\right|,$$
with
$$j=\min\left\{ \left|C_{i}-P_{i}\right|\right\},$$
and finally $r$ is the constant risk-free short rate appropriate for maturity $T$.

## Import modules

In [1]:
import datetime as dt
import pandas as pd
import numpy as np

## Import yields

In the white paper it is assumed that the risk-free rate is 0.38% for both near- and nex-term options. The date chosen for computation is Jan 1, 2009 with options expiring in 9 and 37 days.

In [2]:
f = lambda x: dt.datetime.strptime(x, '%Y%m%d')
yields = pd.read_csv('../data/yields.csv', converters = {'Date' : f})
yields = yields.set_index(['Date','Days'])

yields

Rate
Date       Days      
2009-01-01 9     0.38
           37    0.38

## Import options

In [3]:
# Function to parse dates of '20090101' format
f = lambda x: dt.datetime.strptime(x, '%Y%m%d')
raw_options = pd.read_csv('../data/options.csv', converters = {'Expiration' : f})

# Function to convert days to internal timedelta format
f_delta = lambda x: dt.timedelta(days = int(x))
raw_options['Date'] = raw_options['Expiration'] - raw_options['Days'].map(f_delta)
# Convert integer strikes to float! Otherwise it may lead to accumulation of errors.
raw_options['Strike'] = raw_options['Strike'].astype(float)

raw_options

,Expiration,Days,Strike,Call Bid,Call Ask,Put Bid,Put Ask,Date
0,2009-01-10,9,200.0,717.6,722.80,0.0,0.05,2009-01-01
1,2009-01-10,9,250.0,667.6,672.90,0.0,0.05,2009-01-01
2,2009-01-10,9,300.0,617.9,622.90,0.0,0.05,2009-01-01
3,2009-01-10,9,350.0,567.9,572.90,0.0,0.05,2009-01-01
4,2009-01-10,9,375.0,542.9,547.90,0.0,0.10,2009-01-01
...,...,...,...,...,...,...,...,...
363,2009-02-07,37,1800.0,0.0,0.75,875.6,880.60,2009-01-01
364,2009-02-07,37,1850.0,0.0,0.75,925.5,930.50,2009-01-01
365,2009-02-07,37,1900.0,0.0,0.20,974.7,979.90,2009-01-01
366,2009-02-07,37,1950.0,0.0,0.75,1025.1,1030.60,2009-01-01


## Do some cleaning and indexing

In [4]:
# Since VIX is computed for the date of option quotations, we do not really need Expiration
options = raw_options.set_index(['Date','Days','Strike']).drop('Expiration', axis = 1)

# Do some renaming and separate calls from puts
calls = options[['Call Bid','Call Ask']].rename(columns = {'Call Bid' : 'Bid', 'Call Ask' : 'Ask'})
puts = options[['Put Bid','Put Ask']].rename(columns = {'Put Bid' : 'Bid', 'Put Ask' : 'Ask'})

# Add a column indicating the type of the option
calls['CP'], puts['CP'] = 'C', 'P'

# Merge calls and puts
options = pd.concat([calls, puts])

# Reindex and sort
options = options.reset_index().set_index(['Date','Days','CP','Strike']).sort_index()

options.head()

Bid    Ask
Date       Days CP Strike              
2009-01-01 9    C  200.0   717.6  722.8
                   250.0   667.6  672.9
                   300.0   617.9  622.9
                   350.0   567.9  572.9
                   375.0   542.9  547.9

## Compute bid/ask average

This step is used further to filter out in-the-money options.

In [5]:
options['Premium'] = (options['Bid'] + options['Ask']) / 2
options2 = options[options['Bid'] > 0]['Premium'].unstack('CP')

options2.dropna().head()

CP                           C      P
Date       Days Strike               
2009-01-01 9    400.0   520.45  0.125
                425.0   496.10  0.125
                450.0   470.50  0.125
                470.0   450.50  0.150
                475.0   445.50  0.150

## Determine minimum difference

In [6]:
# Find the absolute difference
options2['CPdiff'] = (options2['C'] - options2['P']).abs()
# Mark the minimum for each date/term
options2['min'] = options2['CPdiff'].groupby(level = ['Date','Days']).transform(lambda x: x == x.min())

options2.dropna().head()

CP                           C      P   CPdiff    min
Date       Days Strike                               
2009-01-01 9    400.0   520.45  0.125  520.325  False
                425.0   496.10  0.125  495.975  False
                450.0   470.50  0.125  470.375  False
                470.0   450.50  0.150  450.350  False
                475.0   445.50  0.150  445.350  False

## Compute forward price

In [7]:
# Leave only at-the-money optons
df = options2[options2['min'] == 1].reset_index()
# Merge with risk-free rate
df = pd.merge(df, yields.reset_index(), how = 'left')

# Compute the implied forward
df['Forward'] = df['CPdiff'] * np.exp(df['Rate'] * df['Days'] / 36500)
df['Forward'] += df['Strike']
forward = df.set_index(['Date','Days'])[['Forward']]

forward.head()

Forward
Date       Days            
2009-01-01 9     920.500047
           37    921.000385

## Compute at-the-money strike

In [8]:
# Merge options with implied forward price
left = options2.reset_index().set_index(['Date','Days'])
df = pd.merge(left, forward, left_index = True, right_index = True)
# Compute at-the-money strike
mid_strike = df[df['Strike'] < df['Forward']]['Strike'].groupby(level = ['Date','Days']).max()
mid_strike = pd.DataFrame({'Mid Strike' : mid_strike})

mid_strike.head()

Mid Strike
Date       Days            
2009-01-01 9          920.0
           37         920.0

## Separate out-of-the-money calls and puts

In [9]:
# Go back to original data and reindex it
left = options.reset_index().set_index(['Date','Days']).drop('Premium', axis = 1)
# Merge with at-the-money strike
df = pd.merge(left, mid_strike, left_index = True, right_index = True)
# Separate out-of-the-money calls and puts
P = (df['Strike'] <= df['Mid Strike']) & (df['CP'] == 'P')
C = (df['Strike'] >= df['Mid Strike']) & (df['CP'] == 'C')
puts, calls = df[P], df[C]

puts.tail()

CP  Strike   Bid   Ask  Mid Strike
Date       Days                                   
2009-01-01 37    P   900.0  50.2  55.4       920.0
           37    P   905.0  52.2  57.2       920.0
           37    P   910.0  54.0  59.5       920.0
           37    P   915.0  56.3  61.5       920.0
           37    P   920.0  57.8  63.3       920.0

In [10]:
calls.head()

CP  Strike   Bid   Ask  Mid Strike
Date       Days                                   
2009-01-01 9     C   920.0  35.2  39.1       920.0
           9     C   925.0  31.4  35.2       920.0
           9     C   930.0  31.0  33.9       920.0
           9     C   935.0  26.0  31.5       920.0
           9     C   940.0  26.0  29.0       920.0

## Remove all quotes after two consecutive zero bids

In [11]:
# Indicator of zero bid
calls = calls.assign(zero_bid=lambda df: (df['Bid'] == 0).astype(int))
# Accumulate number of zero bids starting at-the-money
calls['zero_bid_accum'] = calls.groupby(level = ['Date','Days'])['zero_bid'].cumsum()

# Sort puts in reverse order inside date/term
puts = puts.groupby(level = ['Date','Days']).apply(lambda x: x.sort_values(['Strike'], ascending = False))
# # Indicator of zero bid
puts = puts.assign(zero_bid=lambda df: (df['Bid'] == 0).astype(int))
# # Accumulate number of zero bids starting at-the-money
puts['zero_bid_accum'] = puts.groupby(level = ['Date','Days'])['zero_bid'].cumsum()
# 
calls[(calls['Strike'] >= 1210) & (calls['Strike'] <= 1240)].head()

CP  Strike   Bid  Ask  Mid Strike  zero_bid  zero_bid_accum
Date       Days                                                            
2009-01-01 9     C  1210.0  0.05  0.5       920.0         0               0
           9     C  1215.0  0.05  0.5       920.0         0               0
           9     C  1220.0  0.05  1.0       920.0         0               0
           9     C  1225.0  0.00  1.0       920.0         1               1
           9     C  1230.0  0.00  1.0       920.0         1               2

In [12]:
# Merge puts and cals
options3 = pd.concat([calls, puts]).reset_index()
# Throw away bad stuff
options3 = options3[(options3['zero_bid_accum'] < 2) & (options3['Bid'] > 0)]

# Compute option premium as bid/ask average
options3['Premium'] = (options3['Bid'] + options3['Ask']) / 2
options3 = options3.set_index(['Date','Days','CP','Strike'])['Premium'].unstack('CP')

options3.dropna().head()

CP                          C      P
Date       Days Strike              
2009-01-01 9    920.0   37.15  36.65
           37   920.0   61.55  60.55

## Compute out-of-the-money option price

In [13]:
# Merge wth at-the-money strike price
left = options3.reset_index().set_index(['Date','Days'])
df = pd.merge(left, mid_strike, left_index = True, right_index = True)

# Conditions to separate out-of-the-money puts and calls
condition1 = df['Strike'] < df['Mid Strike']
condition2 = df['Strike'] > df['Mid Strike']
# At-the-money we have two quotes, so take the average
df['Premium'] = (df['P'] + df['C']) / 2
# Remove in-the-money options
df.loc[condition1, 'Premium'] = df.loc[condition1, 'P']
df.loc[condition2, 'Premium'] = df.loc[condition2, 'C']

options4 = df[['Strike','Mid Strike','Premium']].copy()

options4[(options4['Strike'] >= 910) & (options4['Strike'] <= 930)].head()

Strike  Mid Strike  Premium
Date       Days                             
2009-01-01 9      910.0       920.0    31.70
           9      915.0       920.0    33.55
           9      920.0       920.0    36.90
           9      925.0       920.0    33.30
           9      930.0       920.0    32.45

## Compute difference between adjoining strikes

In [14]:
def compute_adjoining_strikes_diff(group):
    new = group.copy()
    new.iloc[1:-1] = np.array((group.values[2:] - group.values[:-2]) / 2)
    new.iloc[0] = group.values[1] - group.values[0]
    new.iloc[-1] = group.values[-1] - group.values[-2]
    return new

options4['dK'] = options4.groupby(['Date','Days'])['Strike'].transform(compute_adjoining_strikes_diff)

options4.head()

Strike  Mid Strike  Premium    dK
Date       Days                                   
2009-01-01 9      400.0       920.0    0.125  25.0
           9      425.0       920.0    0.125  25.0
           9      450.0       920.0    0.125  22.5
           9      470.0       920.0    0.150  12.5
           9      475.0       920.0    0.150   5.0

## Compute contribution of each strike

In [15]:
# Merge with risk-free rate
contrib = pd.merge(options4, yields, left_index = True, right_index = True).reset_index()

contrib['sigma2'] = contrib['dK'] / contrib['Strike'] ** 2
contrib['sigma2'] *= contrib['Premium'] * np.exp(contrib['Rate'] * contrib['Days'] / 36500)

contrib.head()

,Date,Days,Strike,Mid Strike,Premium,dK,Rate,sigma2
0,2009-01-01,9,400.0,920.0,0.125,25.0,0.38,0.000020
1,2009-01-01,9,425.0,920.0,0.125,25.0,0.38,0.000017
2,2009-01-01,9,450.0,920.0,0.125,22.5,0.38,0.000014
3,2009-01-01,9,470.0,920.0,0.150,12.5,0.38,0.000008
4,2009-01-01,9,475.0,920.0,0.150,5.0,0.38,0.000003


## Compute each preiod index

In [16]:
# Sum up contributions from all strikes
sigma2 = contrib.groupby(['Date','Days'])[['sigma2']].sum() * 2

# Merge at-the-money strike and implied forward
sigma2['Mid Strike'] = mid_strike
sigma2['Forward'] = forward

# Compute variance for each term
sigma2['sigma2'] -= (sigma2['Forward'] / sigma2['Mid Strike'] - 1) ** 2
sigma2['sigma2'] /= sigma2.index.get_level_values(1).astype(float) / 365
sigma2 = sigma2[['sigma2']]

sigma2.head()

sigma2
Date       Days          
2009-01-01 9     0.472767
           37    0.366818

## Compute interpolated index

In [17]:
# This function determines near- and next-term if there are several maturities in the data
def f(group):
    days = np.array(group['Days'])
    sigma2 = np.array(group['sigma2'])
    
    if days.min() <= 30:
        T1 = days[days <= 30].max()
    else:
        T1 = days.min()
    
    T2 = days[days > T1].min()
    
    sigma_T1 = sigma2[days == T1][0]
    sigma_T2 = sigma2[days == T2][0]
    
    return pd.DataFrame([{'T1' : T1, 'T2' : T2, 'sigma2_T1' : sigma_T1, 'sigma2_T2' : sigma_T2}])

two_sigmas = sigma2.reset_index().groupby('Date').apply(f, include_groups=False).groupby(level = 'Date').first()

two_sigmas.head()

,T1,T2,sigma2_T1,sigma2_T2
Date,,,,
2009-01-01,9,37,0.472767,0.366818


## Interpolate the VIX

In [18]:
df = two_sigmas.copy()

for t in ['T1','T2']:
    # Convert to fraction of the year
    df['days_' + t] = df[t].astype(float) / 365
    # Convert to miutes
    df[t] = (df[t] - 1) * 1440. + 510 + 930

df['sigma2_T1'] = df['sigma2_T1'] * df['days_T1'] * (df['T2'] - 30. * 1440.)
df['sigma2_T2'] = df['sigma2_T2'] * df['days_T2'] * (30. * 1440. - df['T1'])
df['VIX'] = ((df['sigma2_T1'] + df['sigma2_T2']) / (df['T2'] - df['T1']) * 365. / 30.) ** .5 * 100

VIX = df[['VIX']]

VIX.head()

,VIX
Date,
2009-01-01,61.217999
